In [7]:
from selenium import webdriver
import os
from SICOIN_scraper import *

In [5]:
options = webdriver.ChromeOptions()
options.headless = False # True
# options.add_argument("download.default_directory="+os.getcwd())
options.add_experimental_option("prefs", {
  "download.default_directory": os.path.abspath("../../../Resource Tracking/SICOIN Scraped"),
  "download.prompt_for_download": False,
})
driver = webdriver.Chrome(chrome_options=options)
driver.get('https://sicoin.minfin.gob.gt')

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [8]:
login(driver)
#driver.close()

In [70]:
# IGRC01 - Ejecución Financiera
# TB
for year in range(2012, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink100"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "17"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh

In [ ]:
# IGRC03 - Comparacion de Ejecucion Financiera y Fisica
# TB
for year in range(2012, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink102"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "17"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh()

In [69]:
# IGRC02 - Ejecución Física
# TB
for year in range(2012, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink101"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "17"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh()

In [71]:
# IGRC00, IGRC01, IGRC02
# Malaria
for year in range(2013, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink100"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "18"
            }
        ])
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink101"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "18"
            }
        ])
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink102"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "18"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh()

In [ ]:
# IGRC00, IGRC01, IGRC02
# HIV
for year in range(2013, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink100"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "16"
            }
        ])
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink101"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "16"
            }
        ])
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink102"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "16"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh()